In [19]:
import os
import json
import math
import numpy as np
import copy
from scipy.spatial.transform import Rotation

In [20]:
SAVE_PATH = os.path.join(os.getcwd(), '../data/jackal_training_data')
if not os.path.exists(SAVE_PATH):
    os.makedirs(SAVE_PATH)
if not os.path.exists(os.path.join(SAVE_PATH, 'images')):
    os.makedirs(os.path.join(SAVE_PATH, 'images'))

In [21]:
transforms = {}
transforms["focal_length"] = 4.74 #Focal Length in mm
transforms["fov_x"] = 1.1519 #FOV in radians
transforms["w"] = 640 # image width
transforms["h"] = 420 # image height


transforms["pixel_width"] = 2*math.tan(transforms["fov_x"]) * transforms["focal_length"] / transforms["w"] #Sensor width in mm
transforms["pixel_height"] = transforms["pixel_width"] #Sensor height in mm
transforms["fl_x"] = transforms["focal_length"]/transforms["pixel_width"] # focal length/ pixel_width
transforms["fl_y"] = transforms["focal_length"]/transforms["pixel_height"]# focal length/ pixel_height
transforms["cx"] = transforms["w"]//2 # optical center in x
transforms["cy"] = transforms["h"]//2 # optical center in y
transforms["camera_model"] = "OPENCV" # camera model

ground_truth = {}
ground_truth = transforms.copy()

In [22]:
frames = []
ground_truth_frames = []

starting_pts = [[0.0,-5], [0.0, -4], [0, -3], [0,-2], [0, -1]]
z_store = [2]
pitch = 0.785
spin = 30

cnt = 0

sd = 0.1

for x, y in starting_pts:
    for z in z_store:
        start_transform_matrix = np.eye(4)
        start_transform_matrix[:3, :3] = Rotation.from_rotvec(np.array([pitch, 0, 0])).as_matrix()
        start_transform_matrix[0, 3] = x
        start_transform_matrix[1, 3] = y
        start_transform_matrix[2, 3] = z

        for i in range(spin):
            angle = i * 2 * np.pi / spin
            rot = Rotation.from_rotvec(np.array([0, 0, 1]) * angle).as_matrix()
            rot_matrix = np.eye(4)
            rot_matrix[:3, :3] = rot
            transform_matrix = np.matmul(rot_matrix, start_transform_matrix)

            frame = {}
            file_path = "images/{}.png".format(cnt)

            frame["file_path"] = file_path
            frame["transform_matrix"] = transform_matrix.tolist().copy()
            frames.append(frame.copy())

            
            noise_matrix = np.eye(4)
            noise_matrix[0, 3] = np.random.normal(0, var)
            noise_matrix[1, 3] = np.random.normal(0, var)
            noise_matrix[:3, :3] = Rotation.from_rotvec(np.array([0, 0, var]) * np.random.normal(0, 1)).as_matrix()
            print(transform_matrix)
            transform_matrix = np.matmul(noise_matrix, transform_matrix)
            print(transform_matrix)
            
            frame["transform_matrix"] = transform_matrix.tolist().copy()
            
            ground_truth_frames.append(frame.copy())
            
            
            
            cnt += 1


transforms["frames"] = frames
ground_truth["frames"] = ground_truth_frames

[[ 1.          0.          0.          0.        ]
 [ 0.          0.70738827 -0.70682518 -5.        ]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 9.99998586e-01  1.18940763e-03 -1.18846085e-03 -8.83563676e-03]
 [-1.68140706e-03  7.07387269e-01 -7.06824182e-01 -5.00356757e+00]
 [ 0.00000000e+00  7.06825181e-01  7.07388269e-01  2.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
[[ 0.9781476  -0.14707429  0.14695722  1.03955845]
 [ 0.20791169  0.69193014 -0.69137936 -4.890738  ]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.97819343 -0.1469217   0.14680475  1.02609091]
 [ 0.20769598  0.69196256 -0.69141175 -4.87592434]
 [ 0.          0.70682518  0.70738827  2.        ]
 [ 0.          0.          0.          1.        ]]
[[ 0.91354546 -0.28772073  0.2874917   2.03368322]
 [ 0.40673664  0.64623134 -0.64571693 -4.56772729]
 [ 0.         

In [23]:
print(transforms["frames"])
print(ground_truth["frames"])

[{'file_path': 'images/0.png', 'transform_matrix': [[0.9999999999999999, 0.0, 0.0, 0.0], [0.0, 0.7073882691671997, -0.706825181105366, -5.0], [0.0, 0.706825181105366, 0.7073882691671997, 2.0], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/1.png', 'transform_matrix': [[0.9781476007338055, -0.1470742911072007, 0.14695721851618557, 1.0395584540887963], [0.20791169081775926, 0.6919301382731358, -0.6913793550364513, -4.890738003669028], [0.0, 0.7068251811053659, 0.7073882691671995, 1.9999999999999998], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/2.png', 'transform_matrix': [[0.913545457642601, -0.2877207299522674, 0.2874917014042411, 2.033683215379001], [0.40673664307580015, 0.6462313400873569, -0.645716933546216, -4.567727288213005], [0.0, 0.7068251811053661, 0.7073882691671998, 2.0000000000000004], [0.0, 0.0, 0.0, 1.0]]}, {'file_path': 'images/3.png', 'transform_matrix': [[0.8090169943749472, -0.41579239226117826, 0.4154614174026905, 2.938926261462365], [0.5877852522924729, 0.5722891313

In [24]:
#Save transforms
with open(os.path.join(SAVE_PATH,'transforms.json'), 'w') as outfile:
    json.dump(transforms, outfile, indent=4)
with open(os.path.join(SAVE_PATH,'ground_truth.json'), 'w') as outfile:
    json.dump(ground_truth, outfile, indent=4)